In [1]:
from nltk.sem import Expression
import nltk.sem.logic as L

In [11]:
def eliminate_implication(expr):
    if isinstance(expr, L.ImpExpression):
        return L.OrExpression(L.NegatedExpression(eliminate_implication(expr.first)), eliminate_implication(expr.second))
    elif isinstance(expr, L.BinaryExpression):
        return expr.__class__(eliminate_implication(expr.first), eliminate_implication(expr.second))
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(eliminate_implication(expr.term))
    elif isinstance(expr, L.QuantifiedExpression):
        return expr.__class__(expr.variable, eliminate_implication(expr.term))
    else:
        return expr

In [12]:
expr = Expression.fromstring('((P -> R) -> (Q -> Z))')
print(eliminate_implication(expr))

(-(-P | R) | -Q | Z)


In [14]:
def move_negation_inward(expr): # Demorgan Law
    if isinstance(expr, L.NegatedExpression):
        if isinstance(expr.term, L.AndExpression):
            return L.OrExpression(move_negation_inward(L.NegatedExpression(expr.term.first)), move_negation_inward(L.NegatedExpression(expr.term.second)))
        elif isinstance(expr.term, L.OrExpression):
            return L.AndExpression(move_negation_inward(L.NegatedExpression(expr.term.first)), move_negation_inward(L.NegatedExpression(expr.term.second)))
        elif isinstance(expr.term, L.NegatedExpression):
            return move_negation_inward(expr.term.term)
        else:
            return expr
    elif isinstance(expr, L.BinaryExpression):
        return expr.__class__(move_negation_inward(expr.first), move_negation_inward(expr.second))
    elif isinstance(expr, L.QuantifiedExpression):
        return expr.__class__(expr.variable, move_negation_inward(expr.term))
    else:
        return expr

In [15]:
expr = Expression.fromstring('(-(-P | R) | -Q | Z)')
print(move_negation_inward(expr))

((P & -R) | -Q | Z)


In [16]:
def remove_double_negation(expr):
    if isinstance(expr, L.NegatedExpression):
        if isinstance(expr.term, L.NegatedExpression):
            return remove_double_negation(expr.term.term)
        else:
            return L.NegatedExpression(remove_double_negation(expr.term))
    elif isinstance(expr, L.BinaryExpression):
        return expr.__class__(remove_double_negation(expr.first), remove_double_negation(expr.second))
    elif isinstance(expr, L.QuantifiedExpression):
        return expr.__class__(expr.variable, remove_double_negation(expr.term))
    else:
        return expr

In [17]:
expr = Expression.fromstring('all x. --(P(x) & Q(x))')
print(remove_double_negation(expr))

all x.(P(x) & Q(x))


In [21]:
def standardize_variable_scope(expr, counter=1):
    if isinstance(expr, L.QuantifiedExpression):
        old_var = expr.variable
        new_var = L.Variable(expr.variable.name + str(counter))
        counter += 1
        return expr.__class__(new_var, expr.term.replace(old_var, L.VariableExpression(new_var)))
    elif isinstance(expr, L.ApplicationExpression):
        return L.ApplicationExpression(standardize_variable_scope(expr.function, counter), [standardize_variable_scope(arg, counter) for arg in expr.args])
    elif isinstance(expr, L.BinaryExpression):
        return expr.__class__(standardize_variable_scope(expr.first, counter), standardize_variable_scope(expr.second, counter + 1))
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(standardize_variable_scope(expr.term, counter))
    else:
        return expr

In [31]:
expr = Expression.fromstring('exists x. P(x) | (all x. (Q(x) & R(x)))')
print(standardize_variable_scope(expr))

(exists x1.P(x1) | all x2.(Q(x2) & R(x2)))


In [36]:
def prenex_form(expr):
    if isinstance(expr, L.BinaryExpression):
        first = prenex_form(expr.first)
        second = prenex_form(expr.second)
        if isinstance(first, L.QuantifiedExpression) and not isinstance(second, L.QuantifiedExpression):
            return first.__class__(first.variable, expr.__class__(first.term, second))
        elif not isinstance(first, L.QuantifiedExpression) and isinstance(second, L.QuantifiedExpression):
            return second.__class__(second.variable, expr.__class__(first, second.term))
        elif isinstance(first, L.QuantifiedExpression) and isinstance(second, L.QuantifiedExpression):
            return first.__class__(first.variable, prenex_form(expr.__class__(first.term, second)))
        else:
            return expr
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(prenex_form(expr.term))
    elif isinstance(expr, L.QuantifiedExpression):
        return expr.__class__(expr.variable, prenex_form(expr.term))
    else:
        return expr

In [37]:
expr = Expression.fromstring('(all x. P(x)) | (all y. Q(y))')
print(prenex_form(expr))

all x y.(P(x) | Q(y))


In [38]:
def skolemization(expr, counter=1):
    if isinstance(expr, L.ExistsExpression):
        new_func = L.FunctionVariableExpression(L.Variable('f' + str(counter)))
        counter += 1
        return skolemization(expr.term.replace(expr.variable, new_func), counter)
    elif isinstance(expr, L.BinaryExpression):
        return expr.__class__(skolemization(expr.first, counter), skolemization(expr.second, counter))
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(skolemization(expr.term, counter))
    elif isinstance(expr, L.QuantifiedExpression):
        return expr.__class__(expr.variable, skolemization(expr.term, counter))
    else:
        return expr

In [39]:
expr = Expression.fromstring('exists x. (P(x) | all y. (Q(y) -> R(x, y)))')
print(skolemization(expr))

(P(f1) | all y.(Q(y) -> R(f1,y)))


In [40]:
def eliminate_universal_quantifiers(expr):
    if isinstance(expr, L.AllExpression):
        return eliminate_universal_quantifiers(expr.term)
    elif isinstance(expr, L.BinaryExpression):
        return expr.__class__(eliminate_universal_quantifiers(expr.first), eliminate_universal_quantifiers(expr.second))
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(eliminate_universal_quantifiers(expr.term))
    else:
        return expr

In [41]:
expr = Expression.fromstring('all x. (P(x) -> all y. Q(y))')
print(eliminate_universal_quantifiers(expr))

(P(x) -> Q(y))


In [42]:
def convert_to_cnf(expr):
    if isinstance(expr, L.AndExpression):
        return L.AndExpression(convert_to_cnf(expr.first), convert_to_cnf(expr.second))
    elif isinstance(expr, L.OrExpression):
        first = convert_to_cnf(expr.first)
        second = convert_to_cnf(expr.second)
        if isinstance(first, L.AndExpression):
            return L.AndExpression(L.OrExpression(first.first, second), L.OrExpression(first.second, second))
        elif isinstance(second, L.AndExpression):
            return L.AndExpression(L.OrExpression(first, second.first), L.OrExpression(first, second.second))
        else:
            return expr
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(convert_to_cnf(expr.term))
    elif isinstance(expr, L.QuantifiedExpression):
        return expr.__class__(expr.variable, convert_to_cnf(expr.term))
    else:
        return expr

In [43]:
expr = Expression.fromstring('P | (Q & R)')
print(convert_to_cnf(expr))

((P | Q) & (P | R))


In [44]:
def turn_conjunctions_into_clauses(expr):
    if isinstance(expr, L.AndExpression):
        return turn_conjunctions_into_clauses(expr.first) | turn_conjunctions_into_clauses(expr.second)
    else:
        return {expr}

In [45]:
expr = Expression.fromstring('(P | Q) & (R | S)')
print(turn_conjunctions_into_clauses(expr))

{<OrExpression (R | S)>, <OrExpression (P | Q)>}


In [60]:
def rename_variables(expr, counter=-1):
    if isinstance(expr, L.ApplicationExpression):
        return L.ApplicationExpression(expr.function, rename_variables(expr.argument, counter))
    elif isinstance(expr, L.IndividualVariableExpression):
        return L.IndividualVariableExpression(L.Variable(expr.variable.name + str(counter)))
    elif isinstance(expr, L.BinaryExpression):
        counter += 1
        return expr.__class__(rename_variables(expr.first, counter), rename_variables(expr.second, counter))
    elif isinstance(expr, L.NegatedExpression):
        return L.NegatedExpression(rename_variables(expr.term, counter))
    else:
        return expr

In [67]:
expr = Expression.fromstring('P(x) & Q(y) & (T(u) & (R(z) | S(w)))')
print(rename_variables(expr))

(P(x1) & Q(y1) & T(u1) & (R(z2) | S(w2)))


In [48]:
def resolution_procedure(expr):
    expr = eliminate_implication(expr)
    expr = move_negation_inward(expr)
    expr = remove_double_negation(expr)
    expr = standardize_variable_scope(expr)
    expr = prenex_form(expr)
    expr = skolemization(expr)
    expr = eliminate_universal_quantifiers(expr)
    expr = convert_to_cnf(expr)
    expr = turn_conjunctions_into_clauses(expr)
    expr = rename_variables(expr)

    return expr